In [1]:
%cd D:
%cd ..

d:\MPPT\notebook
d:\MPPT


In [2]:
from MPPT_src.mppt_model import MPPT_Model
from keras.layers import Flatten
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
df = pd.read_excel("dataset/MPPT.xlsx")
df = df.drop(["Lat", "Tahun ", "Provinsi", "Jumlah Penduduk (ribu jiwa)", "Jumlah kemiskinan (sem.2 (september))", "Lon", "Negara"], axis=1)
X = df.iloc[:, :-2]
Y_reg = df.iloc[:,-2]
Y_clf = df.iloc[:,-1]
stdScaler = StandardScaler() # ada nilai minus
minmaxScaler = MinMaxScaler() # tanpa nilai minus
X_stdScaled = stdScaler.fit_transform(X)
X_minmaxScaled = minmaxScaler.fit_transform(X)

In [4]:
endemisitas_class = ["Eliminasi", "Rendah", "Sedang", "Tinggi"]

In [5]:
X_train, X_test, y_clf_train, y_clf_test = train_test_split(X_minmaxScaled, Y_clf, test_size=0.1, random_state=42)
X_train, X_test, y_reg_train, y_reg_test = train_test_split(X_minmaxScaled, Y_reg, test_size=0.2, random_state=42)

## ENDEMISITAS

In [6]:
end_mppt = MPPT_Model(predict_model='END')
end_mlp = end_mppt.load_mppt_mlp('model\mlp_cls\end_model.h5')
end_ml = end_mppt.load_mppt_ml('model\ml_cls\DTc.pkl')

2023-03-07 19:34:05.563 | INFO     | MPPT_src.mppt_model:load_mppt_mlp:182 - model mlp mppt model\mlp_cls\end_model.h5 loaded!
2023-03-07 19:34:06.334 | INFO     | MPPT_src.mppt_model:load_mppt_ml:318 - ml best model model\ml_cls\DTc.pkl loaded!


In [11]:
def test_end_model(n, mlp, ml):
      test = X_test[n,:]
      flatten = Flatten()
      y = endemisitas_class[y_clf_test.values[n]]
      print('class: ', y)
      print('ML_predict: ', end_mppt.mppt_ml_predict(test, ml))
      print('MLP predict: ', end_mppt.mppt_mlp_predict(tf.reshape(flatten(test), (1, 15)), mlp))
      
test_end_model(3, end_mlp, end_ml)

2023-03-07 19:35:19.534 | INFO     | MPPT_src.mppt_model:mppt_ml_predict:333 - Hasil prediksi Endimisitas Malaria -> Sedang


class:  Sedang
ML_predict:  status: Sedang
1/1 [==============================] - 0s 400ms/step


2023-03-07 19:35:20.128 | INFO     | MPPT_src.mppt_model:mppt_mlp_predict:202 - Hasil prediksi Endimisitas Malaria -> Sedang


MLP predict:  status: Sedang


## Annual Paracite Incidence (API)

In [9]:
api_mppt = MPPT_Model(predict_model='API')
api_mlp = api_mppt.load_mppt_mlp('model\\mlp_reg\\api_model.h5')
api_ml = api_mppt.load_mppt_ml('model\ml_reg\RFr.pkl')

2023-03-07 19:34:59.112 | INFO     | MPPT_src.mppt_model:load_mppt_mlp:182 - model mlp mppt model\mlp_reg\api_model.h5 loaded!
2023-03-07 19:35:00.619 | INFO     | MPPT_src.mppt_model:load_mppt_ml:318 - ml best model model\ml_reg\RFr.pkl loaded!


In [13]:
def test_end_model(n, mlp, ml):
      test = X_test[n,:]
      flatten = Flatten()
      y = y_reg_test.values[n]
      print('class: ', y)
      print('ML_predict: ', api_mppt.mppt_ml_predict(test, ml))
      print('MLP predict: ', api_mppt.mppt_mlp_predict(tf.reshape(test, (1, 15)), mlp))
      
test_end_model(15, api_mlp, api_ml)

2023-03-07 19:45:28.612 | INFO     | MPPT_src.mppt_model:mppt_ml_predict:337 - Hasil prediksi API Malaria -> 3.1136799999999987


[1.31736527e-01 7.34052705e-05 1.25391850e-02 5.36258158e-01
 5.06446991e-01 2.35665366e-02 5.68965517e-01 6.21973929e-01
 4.01960784e-01 2.19375640e-02 6.04795258e-02 5.48267084e-03
 2.22115910e-01 7.68542172e-01 9.23857868e-01]
class:  3.89
ML_predict:  3.1136799999999987
1/1 [==============================] - 0s 118ms/step


2023-03-07 19:45:28.960 | INFO     | MPPT_src.mppt_model:mppt_mlp_predict:206 - Hasil prediksi API Malaria -> [[12.09923]]


MLP predict:  [[12.09923]]


: 